### CONFIGURAÇÃO/INSTALAÇÃO DO AMBIENTE

In [1]:
import sqlite3
import pandas as pd

In [3]:
!pip install pyspark 
 
from pyspark.sql import SQLContext, SparkSession, functions as F 
from pyspark import SparkFiles
from pyspark.sql.types import * 
 
spark = SparkSession.builder.getOrCreate() 
sql = SQLContext(spark)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 67.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=962f529327fc2f1f97450f924465b620b3f8e7eca1c386a9cace573f112bae7d
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### CRIAÇÃO DE BANCO DE DADOS

#### PARA CRIAR BANCO DE DADOS, VOCÊ DEVE EXECUTAR O COMANDO

~~~python
**nome_conexao_banco** = sqlite3.connect('**nome do arquivo do banco.db**')
~~~

*   Se o banco já existir, ele irá conectar-se ao arquivo e permitirá usar os objetos já existentes.
*   Se o banco não existir, ele será criado

In [2]:
conexao = sqlite3.connect('pedido_exame.db')

In [4]:
arquivo = 'DADOS_EXAMES.csv'
df = pd.read_csv(arquivo, delimiter=";")
df_exames = spark.read.format('csv').options(inferSchema=False, header=True, delimiter=';').load(arquivo)

#### PARA CRIAR TABELAS, VIEWS, PROCEDORES E QUAISQUER OUTROS OBJETOS, VOCÊ DEVE EXECUTAR A INSTRUÇÃO:

~~~python
nome_conexao.execute('''
CREATE TABLE nome_tabela (
  campo1 tipo
);
''')
nome_conexao.commit()
~~~



In [5]:
conexao.execute('''
CREATE TABLE pedido_exame (
  IdUnidadeAtendimento INTEGER,
  Cidade VARCHAR(100),
  Estado VARCHAR(10),
  NumPedido VARCHAR(20),
  IdExame INTEGER,
  Exame VARCHAR(100),
  SiglaExame VARCHAR(100),
  Material VARCHAR(100),
  SetorExame VARCHAR(100),
  PrecoExame FLOAT,
  DataPedido VARCHAR(30),
  QtdExames INTEGER,
  QtdAmostrasColhidas INTERGER,
  DataPrevistaEntregaResultado VARCHAR(30),
  DataLiberacaoResultado VARCHAR(30)
);
''')
conexao.commit()

conexao.execute('''
CREATE TABLE logs_exames (
  IdAlteracao INTEGER PRIMARY KEY AUTOINCREMENT,
  IdUnidadeAtendimento INTEGER,
  Cidade VARCHAR(100),
  Estado VARCHAR(10),
  NumPedido VARCHAR(20),
  IdExame INTEGER,
  Exame VARCHAR(100),
  SiglaExame VARCHAR(100),
  Material VARCHAR(100),
  SetorExame VARCHAR(100),
  PrecoExame FLOAT,
  DataPedido VARCHAR(30),
  QtdExames INTEGER,
  QtdAmostrasColhidas INTERGER,
  DataPrevistaEntregaResultado VARCHAR(30),
  DataLiberacaoResultado VARCHAR(30)
);
''')
conexao.commit()

# ***Criando triggers***

In [6]:
conexao.execute('''
CREATE TRIGGER IF NOT EXISTS __logs_mudanca_pedido_exame_on_delete
AFTER DELETE 
ON pedido_exame

BEGIN
 INSERT INTO logs_exames (IdUnidadeAtendimento,Cidade,Estado,NumPedido,IdExame,Exame,SiglaExame,Material,SetorExame,PrecoExame,DataPedido,QtdExames,QtdAmostrasColhidas,DataPrevistaEntregaResultado,DataLiberacaoResultado)
   VALUES (
   old.IdUnidadeAtendimento,
   old.Cidade,
   old.Estado,
   old.NumPedido,
   old.IdExame,
   old.Exame,
   old.SiglaExame,
   old.Material,
   old.SetorExame,
   old.PrecoExame,
   old.DataPedido,
   old.QtdExames,
   old.QtdAmostrasColhidas,
   old.DataPrevistaEntregaResultado,
   old.DataLiberacaoResultado,
   'delete'
   );
END; 
''')
conexao.commit()

In [7]:
conexao.execute('''
CREATE TRIGGER IF NOT EXISTS __logs_mudanca_pedido_exame_on_update
AFTER UPDATE 
ON pedido_exame

BEGIN
 INSERT INTO logs_exames (IdUnidadeAtendimento,Cidade,Estado,NumPedido,IdExame,Exame,SiglaExame,Material,SetorExame,PrecoExame,DataPedido,QtdExames,QtdAmostrasColhidas,DataPrevistaEntregaResultado,DataLiberacaoResultado)
   VALUES (
   old.IdUnidadeAtendimento,
   old.Cidade,
   old.Estado,
   old.NumPedido,
   old.IdExame,
   old.Exame,
   old.SiglaExame,
   old.Material,
   old.SetorExame,
   old.PrecoExame,
   old.DataPedido,
   old.QtdExames,
   old.QtdAmostrasColhidas,
   old.DataPrevistaEntregaResultado,
   old.DataLiberacaoResultado,
   'update'
   );
END; 
''')
conexao.commit()

In [8]:
conexao.execute('''
CREATE TRIGGER IF NOT EXISTS __logs_mudanca_pedido_exame_on_insert
AFTER INSERT 
ON pedido_exame

BEGIN
 INSERT INTO logs_exames (IdUnidadeAtendimento,Cidade,Estado,NumPedido,IdExame,Exame,SiglaExame,Material,SetorExame,PrecoExame,DataPedido,QtdExames,QtdAmostrasColhidas,DataPrevistaEntregaResultado,DataLiberacaoResultado)
   VALUES (
   new.IdUnidadeAtendimento,
   new.Cidade,
   new.Estado,
   new.NumPedido,
   new.IdExame,
   new.Exame,
   new.SiglaExame,
   new.Material,
   new.SetorExame,
   new.PrecoExame,
   new.DataPedido,
   new.QtdExames,
   new.QtdAmostrasColhidas,
   new.DataPrevistaEntregaResultado,
   new.DataLiberacaoResultado,
   'insert'
   );
END; 
''')
conexao.commit()

#### PARA INSERIR REGISTROS, VOCÊ DEVER EXECUTAR A INSTRUÇÃO:

~~~python
nome_conexao.execute('''
INSERT INTO <nome da tabela> (campo1, campo2, campo3, ..., campoN) VALUES (valor1, valor2, valor3, ..., valorN)
''')

nome_conexao.commit()
~~~

SE ESTIVER CARREGANDO DADOS DE UM ARQUIVO, PODE EXECUTAR A INSTRUÇÃO POR MEIO DE UM LOOP FOR

* Considere a estrutura do DataFrame mencionado a seguir uma estrutura PySpark. Para outras bibliotecas, você deverá adaptá-lo.

~~~python
for linha in dataframe_dados.rdd.toLocalIterador():
  nome_conexao.execute('INSERT INTO <nome da tabela> (campo1, campo2, campo3, ..., campoN) VALUES (' + str(linha.valor1) + ',\'' + str(linha.valor2) + ',\'' + str(linha.valor3) + ',\'') + str(linha...) + ',\'' + str(linha.valorN) + '\')')
  nome_conexao.commit()
~~~

In [9]:
#for row in df_exames.rdd.toLocalIterator(): 
#  conexao_pedido_exame.execute('INSERT INTO pedido_exame (IdUnidadeAtendimento, Cidade, Estado, NumPedido, IdExame, Exame, SiglaExame, Material, SetorExame, PrecoExame, DataPedido, QtdExames, QtdAmostrasColhidas, DataPrevistaEntregaResultado, DataLiberacaoResultado) VALUES (' + str(row.IdUnidadeAtendimento) + ',\'' + str(row.Cidade) + '\',\'' + str(row.Estado) + '\',\'' + str(row.NumPedido) + '\',' + str(row.IdExame) + ',\'' + str(row.Exame) + '\',\'' + str(row.SiglaExame) + '\',\'' + str(row.Material) + '\',\'' + str(row.SetorExame) + '\',\'' + str(row.PrecoExame) + '\',\'' + str(row.DataPedido) + '\',' + str(row.QtdExames) + ',' + str(row.QtdAmostrasColhidas) + ',\'' + str(row.DataPrevistaEntregaResultado) + '\',\'' + str(row.DataLiberacaoResultado) + '\');') 
#  conexao_pedido_exame.commit()

df.to_sql('pedido_exame', conexao, if_exists='replace', index=False)

#### PARA ATUALIZAR DADOS, VOCÊ DEVE EXECUTAR A INSTRUÇÃO:

~~~python
nome_conexao.execute('''
UPDATE <nome da tabela>
SET   campo1 = <novo valor>
WHERE condicao ...
''')

nome_conexao.commit()
~~~



In [10]:
conexao.execute('''
UPDATE pedido_exame
SET Cidade = 'Ipatinga'
WHERE IdExame = 6986
''')

conexao.commit()

#### PARA REMOVER REGISTROS, VOCÊ DEVE EXECUTAR A INSTRUÇÃO:

~~~python
nome_conexao.execute('''
DELETE
FROM  <nome da tabela>
WHERE condicao ...
''')
~~~

In [11]:
conexao.execute('''
DELETE
FROM  pedido_exame
WHERE IdExame = 8397
''')
conexao.commit()

#### PARA SELECIONAR DADOS, VOCÊ DEVE EXECUTAR A INSTRUÇÃO:

~~~python
dados = nome_conexao.execute('''
SELECT ...
''')
nome_conexao.commit()

for linha in dados:
    print(linha)
~~~

In [12]:
dados = conexao.execute('''
select * from pedido_exame
WHERE IdExame = 6986
AND MATERIAL  = 'SANGUE'
''');

conexao.commit()
for linha in dados:
  print(linha)

A saída de streaming foi truncada nas últimas 5000 linhas.
(7, 'Ipatinga', 'MG', 'PED-71478808', 6986, 'TSH ULTRA SENSÍVEL', 'TSH-B', 'SANGUE', 'HORMONIO', '8,35', '2022-06-16 08:05:00', 1.0, 0.0, '2022-06-17 16:00:00', '2022-06-16 21:07:03')
(7, 'Ipatinga', 'MG', 'PED-7264480', 6986, 'TSH ULTRA SENSÍVEL', 'TSH-B', 'SANGUE', 'HORMONIO', '8,35', '2022-06-01 08:08:00', 1.0, 0.0, '2022-06-02 16:00:00', '2022-06-01 19:29:51')
(6, 'Ipatinga', 'MG', 'PED-64167840', 6986, 'TSH ULTRA SENSÍVEL', 'TSH-B', 'SANGUE', 'HORMONIO', '8,35', '2022-06-28 08:32:00', 1.0, 1.0, '2022-06-29 16:00:00', '2022-06-28 17:57:50')
(7, 'Ipatinga', 'MG', 'PED-72025096', 6986, 'TSH ULTRA SENSÍVEL', 'TSH-B', 'SANGUE', 'HORMONIO', '8,35', '2022-06-23 08:37:00', 1.0, 0.0, '2022-06-24 16:00:00', '2022-06-23 19:28:08')
(6, 'Ipatinga', 'MG', 'PED-64969488', 6986, 'TSH ULTRA SENSÍVEL', 'TSH-B', 'SANGUE', 'HORMONIO', '8,35', '2022-06-11 07:53:00', 1.0, 0.0, '2022-06-12 16:00:00', '2022-06-11 22:59:13')
(7, 'Ipatinga', 'MG', 